# Parse Annotations

Join annotations and create dict of dataframes + car counts

In [21]:
from parsingfunc import *

annotations_dir = "../data/VEDAI1024/Annotations1024"
annotations_dict = parse_annotations_to_dict(annotations_dir)


In [22]:
annotations_dict['00000700']

{'annotation_df':      center_x    center_y  orientation  class_label  occluded  \
 0   58.681547  632.567510    -2.844031            1         1   
 1  953.272985  918.473923    -2.807891            1         1   
 
    fully_contained  corner1_x  corner1_y  corner2_x  corner2_y  corner3_x  \
 0                0         41         81         74         39        619   
 1                0        938        976        968        931        906   
 
    corner3_y  corner4_x  corner4_y  
 0        632        645        635  
 1        919        931        918  ,
 'df_length': 2}

# Parse Images

Extract images from Tarfiles

Keep commented - tars have been deleted but images are stored locally

In [23]:
# source_dir = '../raw_img/'
# dest_dir = '../data/VEDAI1024/images/'
# base_filename = 'Vehicules1024'  # Base name of the tar files
# combined_filename = 'Vehicules1024_combined.tar'  # Name for the combined tar file

# # Run the script
# handle_split_tars(source_dir, dest_dir, base_filename, combined_filename)

# Training a Model

In [24]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np

In [25]:
import torch
from torchvision import transforms
from PIL import Image
import os
import numpy as np

class VEDAI_Dataset(torch.utils.data.Dataset):
    def __init__(self, annotations_dict, img_dir, transform=None):
        self.annotations_dict = annotations_dict
        self.img_dir = img_dir
        self.transform = transform if transform is not None else transforms.Compose([
            transforms.Resize((256, 256)),  # Resize the image to 256x256
            transforms.ToTensor()  # Convert the image to a PyTorch tensor
        ])

    def __len__(self):
        return len(self.annotations_dict)

    def __getitem__(self, idx):
        idx = str(idx).zfill(8)
        img_name = os.path.join(self.img_dir, f'{idx}_co.png')
        image = Image.open(img_name).convert("RGB")
        
        original_width, original_height = image.size
        
        if self.transform:
            image = self.transform(image)
        
        # Now image is a tensor, so get its current shape
        _, height, width = image.shape

        scale_x = 256 / original_width
        scale_y = 256 / original_height

        labels = self.annotations_dict[idx]['annotation_df']
        
        # Converting DataFrame to numpy and ensuring it's in float for accurate scaling
        corners = labels[['corner1_x', 'corner1_y', 'corner2_x', 'corner2_y',
                          'corner3_x', 'corner3_y', 'corner4_x', 'corner4_y']].to_numpy().astype(np.float32)
        
        # Apply scaling to corners
        corners[:, [0, 2, 4, 6]] *= scale_x  # Scale x coordinates
        corners[:, [1, 3, 5, 7]] *= scale_y  # Scale y coordinates
        
        # Calculate bounding box [x_min, y_min, x_max, y_max]
        x_min = np.min(corners[:, [0, 2, 4, 6]], axis=1)
        y_min = np.min(corners[:, [1, 3, 5, 7]], axis=1)
        x_max = np.max(corners[:, [0, 2, 4, 6]], axis=1)
        y_max = np.max(corners[:, [1, 3, 5, 7]], axis=1)
        
        boxes = torch.as_tensor(np.stack([x_min, y_min, x_max, y_max], axis=1), dtype=torch.float32)
        labels = torch.ones((len(boxes),), dtype=torch.int64)  # Assuming '1' is the class label for cars
        
        target = {'boxes': boxes, 'labels': labels}
        
        return image, target


In [26]:
dataset = VEDAI_Dataset(annotations_dict, '../data/VEDAI1024/images/Vehicules1024')

In [27]:
import torchvision
from torchvision.models.detection import ssd300_vgg16

num_classes = 2

# Load the pretrained SSD model with custom number of classes
# If starting from pretrained weights, specify the weights and adjust num_classes accordingly
model = ssd300_vgg16(num_classes=num_classes)

In [28]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [29]:
from torch.utils.data import DataLoader, random_split

def train_model(model, dataset, device, epochs=10):
    # Split dataset into training and validation sets
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

    model.to(device)
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

    for epoch in range(epochs):
        running_loss = 0.0
        for images, targets in train_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            losses.backward()
            optimizer.step()
            running_loss += losses.item()

        # Print average loss per epoch
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, targets in val_loader:
                images = list(img.to(device) for img in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                val_loss += losses.item()

        print(f'Validation Loss: {val_loss/len(val_loader)}')
        model.train()

    print("Training complete")

In [30]:
train_model(model, dataset, device, epochs=1)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/samsonbakos/Documents/GitHub.nosync/EVStationDemand/data/VEDAI1024/images/Vehicules1024/00000023_co.png'